### data filtering

#### imports

In [26]:
import pandas as pd
import pandasql as ps
from helpers import get_next_filename 

#### functions

In [27]:
def process_dataset(df):
    """
    Processes the dataset according to the specified requirements:
    1. Filters rows where position contains 'Data Scientist' or 'Data Science'.
    2. Sorts rows by the 'period' field.
    3. Adds a new column 'final_seniority' with the first non-empty value from seniority -> local experience seniority -> general experience seniority.
    4. Adds a new column 'final_salary' as a concatenation of salary, bonuses flag, and bonuses amount.
    5. Repeats rows based on the 'frequency' column value (converted to int). Defaults to 1 if frequency is missing or NaN.
    6. Keeps only the columns: 'period', 'final_seniority', 'final_salary'.

    :param df: Input DataFrame.
    :return: Processed DataFrame.
    """
    # Step 1: Filter rows where position contains 'Data Scientist' or 'Data Science'
    filtered_df = df[df['position'].str.contains('Data Scientist|Data Science', na=False)]

    # Step 2: Sort rows by the 'period' field
    sorted_df = filtered_df.sort_values(by='period')

    # Списки категорій
    junior_values = [
        '1', '1.5', '2', '0.25', 'менше 3 місяців', '0.5', '1 рік', '2 роки', 'півроку', '3 місяці', 
        'менше 3 місяців', '2.0', '1.0', '0.0', 'менше як 3 місяці', 'пів року', '1,5 року', '1-3 роки', 
        'до року', 'junior', 'немає тайтлу', 'intern/trainee', "меньше 3 месяцев"
    ]
    middle_values = [
        '3', '4', '5', '4 роки', '3 роки', '5 років', '5.0', '4.0', '3.0', '4-6 років', 'middle'
    ]
    senior_values = [
        '10 и более лет', '8', '9', '6', '7', '10 і більше років', '7 років', '6 років', '8 років', 'senior', '7.0', '8.0', '9.0', '10.0', '9 років',
        '7-10 років', '10+ років', 'Tech Lead', 'Senior', 'Team Lead', 'Manager', 'Architect','Lead/Team Lead', 'Principal', '6.0', 'head'
    ]
    
    # Функція для визначення категорії
    def map_seniority(value):
        if pd.isna(value):
            return None
        value = str(value).strip().lower()
        if value in [v.lower() for v in junior_values]:
            return 'junior'
        elif value in [v.lower() for v in middle_values]:
            return 'middle'
        elif value in [v.lower() for v in senior_values]:
            return 'senior'
        return value
    
    # Додаємо колонку 'final_seniority'
    sorted_df['final_seniority'] = sorted_df.apply(
        lambda row: next(
            (val for val in [row['seniority'], row['general experience seniority'], row['local experience seniority']] if pd.notna(val)),
            None
        ),
        axis=1
    )
    
    # Оновлюємо значення в колонці 'final_seniority' залежно від категорій
    sorted_df['final_seniority'] = sorted_df['final_seniority'].apply(map_seniority)

    # Додаємо 'final_salary' column
    sorted_df["salary"] = sorted_df.apply(lambda row: process_salary(row["salary"], row.name), axis=1)
    sorted_df['final_salary'] = sorted_df.apply(calculate_final_salary, axis=1)
    processed_df = apply_frequency(sorted_df)
    final_columns = ['period', 'position', "final_seniority", "final_salary"]
    combined_df = processed_df[final_columns]
    
    avg_middle_salary = data1[data1['final_seniority'] == 'middle']['final_salary'].mean()

    def update_seniority(row):
        if row['final_seniority'] == 'senior' and row['final_salary'] < avg_middle_salary:
            return 'junior'
        elif row['final_seniority'] == 'junior' and row['final_salary'] > avg_middle_salary:
            return 'senior'
        else:
            return row['final_seniority']
        
    combined_df.loc[:, 'final_seniority'] = combined_df.apply(update_seniority, axis=1)
    return combined_df

def process_salary(salary, index):
    """
    Processes the salary values:
    - If the value is a range (e.g., "500 - 1000"), replaces it with the midpoint of the range.
    - If the value contains a "+", rounds up to the next whole number.
    - If the value contains a comma instead of a dot (e.g., "3284,69"), replaces the comma with a dot.
    - Converts the value to a float.
    - Logs and removes incorrect values.

    :param salary: The salary value to process.
    :param index: The index of the row being processed.
    :return: Processed salary value as a float or None if invalid.
    """
    if pd.isna(salary):
        print(f"Рядок {index}: значення '{salary}' було видалено через 'NaN (відсутнє значення)'")
        df.drop(index, inplace=True)
        return None
    
    salary = str(salary).strip()
    
    # Обробка діапазонів
    if "-" in salary:
        try:
            # Розділяємо діапазон на дві частини
            low, high = salary.split("-")
            # Перетворюємо частини на float і обчислюємо середину діапазону
            return (float(low.strip()) + float(high.strip())) / 2
        except ValueError:
            print(index, salary, "Некоректний діапазон")
            return None
    
    # Обробка значень із "+"
    if "+" in salary:
        try:
            return float(salary.replace("+", "").strip()) + 500
        except ValueError:
            print(index, salary, "Некоректне значення з '+'")
            return None
    
    # Обробка значень із комою замість крапки
    if "," in salary:
        try:
            salary = salary.replace(",", ".")
            return float(salary)
        except ValueError:
            print(index, salary, "Некоректне значення з ','")
            return None
    
    # Перетворення на float для інших випадків
    try:
        return float(salary)
    except ValueError:
        print(index, salary, "Некоректне значення")
        return None



def calculate_final_salary(row):
    """
    Calculates the final salary based on the value in the 'bonuses flag' column.

    :param row: The row of the DataFrame being processed.
    :return: The calculated final salary as an integer.
    """
    # Якщо bonuses flag має одне з цих значень, залишаємо тільки salary
    if row['bonuses flag'] in ["Отримую бонус за гарні показники роботи, але нерегулярно", None, "Ні, не отримую"]:
        return int(row['salary'])
    
    # Якщо bonuses flag має одне з цих значень, розраховуємо бонуси на місяць
    elif row['bonuses flag'] in ["Отримую бонус щокварталу", "Отримую бонус щомісяця", "Отримую бонус щороку"]:
        # Перевіряємо, чи є значення в bonuses amount (щоб уникнути проблем із NaN)
        if pd.notna(row['bonuses amount']):
            if row['bonuses flag'] == "Отримую бонус щомісяця":
                return int(row['salary'] + float(row['bonuses amount']))
            elif row['bonuses flag'] == "Отримую бонус щокварталу":
                return int(row['salary'] + float(row['bonuses amount']) / 4.0)
            elif row['bonuses flag'] == "Отримую бонус щороку":
                return int(row['salary'] + float(row['bonuses amount']) / 12.0)
        else:
            return int(row['salary'])
    else:
        return int(row['salary'])
def apply_frequency(df):
    """
    Applies the frequency to repeat rows in the DataFrame based on the 'frequency' column.

    :param df: Input DataFrame.
    :return: DataFrame with rows repeated based on the 'frequency' column.
    """
    df['frequency'] = df['frequency'].fillna(1.0)

    df['frequency'] = df['frequency'].apply(lambda x: max(x, 0))

    df['frequency'] = df['frequency'].astype(int)

    df = df.loc[df.index.repeat(df['frequency'])].reset_index(drop=True)
    return df

#### variables

In [28]:
input_path = "raw_salary_compile/combined_salary_1.csv"
output_folder = "filtered_dataset" 

#### main functions

In [29]:
output_path = get_next_filename("filtered_dataset", output_folder)
df = pd.read_csv(input_path, low_memory=False)

processed_df = process_dataset(df)
processed_df.to_csv(output_path, index=False)
print(f"saved file to {output_path}")

saved file to filtered_dataset\filtered_dataset_1.csv


In [30]:
data = pd.read_csv(output_path, low_memory=False)
query = """
    SELECT DISTINCT `final_seniority`
    FROM data
"""

unique_bonuses_flag = ps.sqldf(query, locals())

print(unique_bonuses_flag)

  final_seniority
0          junior
1          senior
2          middle


In [31]:
data1 = pd.read_csv(output_path, low_memory=False)
query = """
    SELECT avg(final_salary)  FROM data 
    where final_seniority = 'middle'
    and period = '2021_H1'
    group by final_seniority
"""

unique_bonuses_flag = ps.sqldf(query, locals())

print(unique_bonuses_flag)

   avg(final_salary)
0        3762.789474
